In [17]:
import requests

In [18]:
import geopandas as gpd
import pandas as pd

## Open State data

In [19]:
file_path = "data/cb_2024_us_county_500k/cb_2024_us_county_500k.shp"
counties_gdf = gpd.read_file(file_path)

In [20]:
counties_gdf = counties_gdf[
    ~counties_gdf["STATEFP"].isin(["60", "66", "69", "72", "74", "78"])
].reset_index(drop=True)

## Get Labor Data

In [21]:
table = "S2301"

url = "https://api.census.gov/data/2023/acs/acs5/subject"
params = {
    "get": f"group({table})",
    "ucgid": "pseudo(0100000US$0500000)",
}
response = requests.get(url, params=params)

In [22]:
data = response.json()
columns = data[0]
rows = data[1:]
labor_df = pd.DataFrame(rows, columns=columns)

In [23]:
url = "http://api.census.gov/data/2023/acs/acs5/subject/variables.json"
response = requests.get(url)
variables = response.json()["variables"]

In [24]:
col_vars = {k: v["label"] for k, v in variables.items() if k.startswith(table)}

In [25]:
labor_df = labor_df.rename(columns=col_vars)
labor_df = labor_df[[*list(col_vars.values()), "ucgid"]]

In [ ]:
rename_dict = {
    col: col.split("!!")[-1]
    for col in labor_df.columns
    if col.startswith(
        "Estimate!!Labor Force Participation Rate!!Population 20 to 64 years!!SEX!!Male"
    )
}

In [29]:
labor_df.rename(columns=rename_dict, inplace=True)

In [ ]:
labor_df = labor_df[["Male", "ucgid"]]
labor_df = labor_df.rename(columns={"ucgid": "GEOIDFQ"})

In [14]:
labor_df["Male"] = labor_df["Male"].astype(float)

## Merge Data

In [15]:
labor_gdf = counties_gdf.merge(labor_df, on="GEOIDFQ", how="inner")

In [16]:
labor_gdf = labor_gdf.to_crs(9311)
labor_gdf.to_file("data/labor_rate_participation.gpkg")